In [49]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import ttest_ind
from dipy.core.sphere import disperse_charges, HemiSphere
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere
from dipy.sims.voxel import multi_tensor, multi_tensor_odf
#from dipy.viz import window, actor
import nibabel as nib

In [50]:
# Open the file in read mode
with open('real_data/hardi-scheme.bvec.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()

# Initialize an empty matrix to store the values
real_bvecs = []

# Iterate through each line
for line in lines:
    # Split the line into individual values
    values = line.split() 
    # Convert the values to integers and append to the matrix
    real_bvecs.append([float(value) for value in values])

# Convert the list of lists into a 2D NumPy array
real_bvecs = np.array(real_bvecs).T

In [51]:
# Open the file in read mode
with open('real_data/hardi-scheme.bval.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()

# Initialize an empty matrix to store the values
real_bvals = []

# Iterate through each line
for line in lines:
    # Split the line into individual values
    values = line.split() 
    # Convert the values to integers and append to the matrix
    real_bvals.append([float(value) for value in values])

# Convert the list of lists into a 2D NumPy array
real_bvals = np.array(real_bvals).T

In [52]:
mask = nib.load('real_data/mask.nii').get_fdata().astype(bool)
white_mask = nib.load('real_data/white_matter_mask.nii').get_fdata().astype(bool)

hardi_snr10 = nib.load('real_data/DWIS_hardi-scheme_SNR-10.nii').get_fdata()
hardi_snr20 = nib.load('real_data/DWIS_hardi-scheme_SNR-20.nii').get_fdata()
hardi_snr30 = nib.load('real_data/DWIS_hardi-scheme_SNR-30.nii').get_fdata()

In [53]:
print(mask.shape)
print(white_mask.shape)
print(hardi_snr10.shape)
print(hardi_snr20.shape)
print(hardi_snr30.shape)

(50, 50, 50)
(50, 50, 50)
(50, 50, 50, 65)
(50, 50, 50, 65)
(50, 50, 50, 65)


In [54]:
hardi_snr10_masked = hardi_snr10[mask]
hardi_snr20_masked = hardi_snr20[mask]
hardi_snr30_masked = hardi_snr30[mask]
hardi_snr10_masked *= (100/hardi_snr10_masked[0][0])
hardi_snr20_masked *= hardi_snr20_masked * (100/hardi_snr20_masked[0][0])
hardi_snr30_masked *= hardi_snr30_masked * (100/hardi_snr30_masked[0][0])

hardi_snr10_white_masked = hardi_snr10[white_mask]
hardi_snr20_white_masked = hardi_snr20[white_mask]
hardi_snr30_white_masked = hardi_snr30[white_mask]
hardi_snr10_white_masked *= (100/hardi_snr10_white_masked[0][0])
hardi_snr20_white_masked *= (100/hardi_snr20_white_masked[0][0])
hardi_snr30_white_masked *= (100/hardi_snr30_white_masked[0][0])

In [55]:
hardi_white_masked_all = np.vstack((hardi_snr10_white_masked, hardi_snr20_white_masked, hardi_snr30_white_masked))
np.random.seed(42)
np.random.shuffle(hardi_white_masked_all)
hardi_white_masked_all

array([[100.        ,  16.68141728,  18.07916501, ...,  45.47868282,
         34.99557482,  33.45439009],
       [100.        ,  59.48667867,  33.93658254, ...,  47.00155644,
         30.4361095 ,  61.85186316],
       [100.        ,   4.21460616,  34.94369335, ...,  25.97735527,
          1.99285867,  11.76183355],
       ...,
       [100.        ,  42.30170598,  43.54686117, ...,  46.70857875,
         51.6891995 ,  49.24161504],
       [100.        ,  44.74013489,  25.11978515, ...,  35.18784143,
          7.30002747,  23.44431898],
       [100.        ,  32.28247932,  31.48289438, ...,  35.86230049,
         12.74147771,  15.92455824]])

In [56]:
S = np.load("synthetic_data/S.npy")
S

array([[101.32133617,   9.7680149 ,  16.03036238, ...,  16.87972262,
         20.07306575,  11.25303949],
       [104.28149415,   5.93050041,  17.23509324, ...,   7.42570262,
         11.71860801,   2.49698402],
       [103.57953331,   7.99781564,  18.99560582, ...,  10.78589485,
         22.75529833,  14.35087365],
       ...,
       [ 96.46219418,  10.66674691,  11.73221267, ...,  19.86385006,
          4.7021142 ,  22.63979334],
       [ 96.11551131,  18.48065961,  31.7608741 , ...,  26.778466  ,
          8.03349262,  13.10915869],
       [ 97.62307686,  15.71742277,   6.7593108 , ...,   4.95291458,
         31.89899917,   0.44614374]])

In [57]:
S_sample_rows = np.random.choice(S.shape[0], size= hardi_white_masked_all.shape[0], replace=False)
S_sampled = S[S_sample_rows, :]

real_row_std = np.std(hardi_white_masked_all, axis=1)
synthetic_row_std = np.std(S_sampled, axis=1)

print(f"Mean of standard deviations of real data: {real_row_std.mean():.3f}")
print(f"Mean of standard deviations of synthetic data: {synthetic_row_std.mean():.3f}")

t_statistic, p_value = ttest_ind(real_row_std, synthetic_row_std)
print(f"p-value of t-test between standard deviations: {p_value:.3f}")

Mean of standard deviations of real data: 12.763
Mean of standard deviations of synthetic data: 12.671
p-value of t-test between standard deviations: 0.000


In [60]:
print(f"Standard deviation of standard deviations for different signals - real: {np.std(real_row_std):.3f}")
print(f"Standard deviation of standard deviations for different signals - synthetic: {np.std(synthetic_row_std):.3f}")

Standard deviation of standard deviations for different signals - real: 2.747
Standard deviation of standard deviations for different signals - synthetic: 0.958
